In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/data/lidar_tmp.pkl', 'rb') as f:
    ds = pickle.load(f)
# ds = ds.isel(time = slice(0, 3))

In [ ]:
__nam

In [ ]:
import sys
sys.path.append('../..')

from spicy_snow.retrieval import retrieve_snow_depth

# log = make_spicy_logger(debug = True)

In [ ]:
log.debug('hello?')